In [1]:
import re
import os
import shutil
import time
import multiprocessing as mp
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

import jieba

In [15]:
base = '../DATA'
filelist = ['PART_I.txt', 'PART_II.txt', 'PART_III.txt']
# filelist = ['sample.txt']

In [3]:
def convert_xml(filename, base=None, converted_file='converted.tmp'):
    # convert file to a valid xml file.
    regex = re.compile(r'(?P<number>\d+)>')
    if base is None:
        fpout = open(converted_file, 'w')
        filepath = filename
    else:
        fpout = open(os.path.join(base, converted_file), 'w')
        filepath = os.path.join(base, filename)
    fpout.write('<root>\n')
    for line in open(filepath):
        txt = regex.sub(lambda x: '"%s">' % x.group('number'), line)
        if '<doc' in txt or '</doc>' in txt or 'summary>' in txt or 'short_text>' in txt or 'human_label>' in txt:
            pass
        else:
            txt = txt.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('""', '"').replace('"', '&quot;').replace("'", '&apos;')
        fpout.write(txt)
    fpout.write('</root>')
    fpout.close()

In [13]:
def _f(doc):
    summary, text = doc
    summary = ' '.join(jieba.cut(summary.text.strip().replace(' ', '').replace('"', "'")))
    text = ' '.join(jieba.cut(text.text.strip().replace(' ', '').replace('"', "'")))
    text = '<d> <p> <s>' + text.replace('。', '。 </s> <s>') + '</s> </p> </d>'
    text = text.replace('<s> </s>', '')
    return dict(summary=summary, text=text)
    
    
def convert_data(filename, base=None, pool=None, converted_file='converted.tmp'):
    if base is None:
        os.makedirs('converted', exist_ok=True)
        fptrain = open(os.path.join('converted', filename), 'w')
    else:
        os.makedirs(os.path.join(base, 'converted'), exist_ok=True)
        fptrain = open(os.path.join(base, 'converted', filename), 'w')
        converted_file = os.path.join(base, converted_file)
    if pool is None:
        pool = mp.Pool(25)
    xml = ET.ElementTree(file=converted_file)
    root = xml.getroot()
    docs = []
    for doc in root:
        docs.append((doc.find('summary'), doc.find('short_text')))
    for res in pool.map(_f, docs):
        fptrain.write("abstract=b'<d> <p> <s> {summary} </s> </p> </d>'\tarticle=b'{text}'\tpublisher=b'HIT'\n".format(**res))
    fptrain.close()

In [7]:
pool = mp.Pool(25)

Process ForkPoolWorker-23:
Process ForkPoolWorker-22:
Process ForkPoolWorker-6:
Process ForkPoolWorker-20:
Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-16:
Process ForkPoolWorker-18:
Process ForkPoolWorker-3:
Process ForkPoolWorker-17:
Process ForkPoolWorker-21:
Process ForkPoolWorker-1:
Process ForkPoolWorker-25:
Process ForkPoolWorker-24:
Process ForkPoolWorker-19:
Process ForkPoolWorker-15:
Process ForkPoolWorker-11:
Process ForkPoolWorker-14:
Process ForkPoolWorker-12:
Process ForkPoolWorker-13:
Traceback (most recent call last):
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/LAB/tangb/anaconda3/lib/python

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", li

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._seml

In [17]:
# convert text to valid xml files and parse it to convert the format.
print('converting...')
for filename in filelist:
    convert_xml(filename, base=base)
    convert_data(filename, base=base)
print('finished!')

converting...


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model cost 1.087 seconds.
Loading model from cache /tmp/jieba.cache
Prefix dict has been built succesfully.
Building prefix dict from the default dictionary ...
Loading model cost 1.224 seconds.
Loading model from cache /tmp/jieba.cache
Prefix dict has been built succesfully.
Building prefix dict from the default dictionary ...
Loading model cost 1.079 seconds.
Prefix dict has been built succesfully.
Prefix dict has been built succesfully.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.167 seconds.
Prefix dict has been built succesfully.
Loading model cost 1.062 seconds.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading

Prefix dict has been built succesfully.
Prefix dict has been built succesfully.
Loading model cost 2.930 seconds.
Prefix dict has been built succesfully.
Loading model cost 2.956 seconds.
Prefix dict has been built succesfully.
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict from the default dictionary ...
Building prefix dict 

finished!


Process ForkPoolWorker-265:
Process ForkPoolWorker-319:
Process ForkPoolWorker-318:
Process ForkPoolWorker-285:
Process ForkPoolWorker-261:
Process ForkPoolWorker-321:
Process ForkPoolWorker-295:
Process ForkPoolWorker-281:
Process ForkPoolWorker-264:
Process ForkPoolWorker-322:
Process ForkPoolWorker-263:
Process ForkPoolWorker-271:
Process ForkPoolWorker-283:
Process ForkPoolWorker-317:
Process ForkPoolWorker-316:
Process ForkPoolWorker-312:
Process ForkPoolWorker-315:
Process ForkPoolWorker-256:
Process ForkPoolWorker-310:
Process ForkPoolWorker-311:
Process ForkPoolWorker-306:
Process ForkPoolWorker-313:
Process ForkPoolWorker-302:
Process ForkPoolWorker-301:
Process ForkPoolWorker-305:
Process ForkPoolWorker-307:
Process ForkPoolWorker-300:
Process ForkPoolWorker-303:
Process ForkPoolWorker-298:
Process ForkPoolWorker-308:
Process ForkPoolWorker-296:
Process ForkPoolWorker-297:
Process ForkPoolWorker-293:
Process ForkPoolWorker-292:
Process ForkPoolWorker-304:
Process ForkPoolWork

...

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
Traceback (most recent call last):
KeyboardInterrupt
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    ta

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._seml

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/pool.py", line 

  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/LAB/tangb/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  

In [18]:
# remove duplicate lines in each file and form train/eval/test file.
fp2 = set(open(os.path.join(base, 'converted', 'PART_II.txt'), 'r').readlines())
fp1 = set(open(os.path.join(base, 'converted', 'PART_I.txt'), 'r').readlines())
fp3 = set(open(os.path.join(base, 'converted', 'PART_III.txt'), 'r').readlines())

In [19]:
# remove lines contained in Part2 from Part1.
fptrain = open(os.path.join(base,'converted', 'train.txt'), 'w')
for line in fp1:
    if line not in fp2:
        fptrain.write(line)
fptrain.close()

In [20]:
open(os.path.join(base,'converted', 'eval.txt'), 'w').writelines(fp2)
open(os.path.join(base,'converted', 'test.txt'), 'w').writelines(fp3)

In [56]:
del fp1, fp2, fp3

In [23]:
# generate vocab file.
corpus = []
with open(os.path.join(base,'converted', 'train.txt')) as f:
    for line in f:
        strs = line.strip().split('\t')
        
        for s in strs:
            if s.startswith('abstract=b\''):
                corpus.append(s.replace('abstract=b\'', '').replace("'", ''))
            if s.startswith('article=b\''):
                corpus.append(s.replace('article=b\'', '').replace("'", ''))

In [24]:
corpus[:5]

['<d> <p> <s> 母亲 街上 卖 烤红薯 女儿 路灯 下 学习 相陪 </s> </p> </d>',
 '<d> <p> <s>21 日 晚上 ， 在 陕西省 榆林市 新建 路 ， 一名 女孩 在 昏暗 的 灯光 下 看书 学习 ， 女孩 的 母亲 在 一旁 卖 烤红薯 ， 这 一幕 被 当地 一名 网友 拍 下发 到 网上 后 引起 当地 网友 关注 ， 网友 纷纷 转载 并 留言 表示 感动 ， 称赞 两 人为 “ 最美 母女 ” 。 </s> <s></s> </p> </d>',
 '<d> <p> <s> 地铁 开进 大学城 吃喝玩乐 快 看 一日游 攻略 </s> </p> </d>',
 '<d> <p> <s>地铁 通 了 ， 大学城 离 你 还 远 吗 ？ 想 不想 换个 方式 ， 抽个 周末 去 大学城 一日游 ? 怎么 玩 、 哪里 好玩 、 需要 多少 钱 、 吃 什么 … … 这些 问题 ， 重庆晚报 记者 都 为 你 打探 好 了 。 </s> <s> 拿 着 这张 报纸 ， 赶紧 开始 你 的 大学城 之旅 吧 ！</s> </p> </d>',
 '<d> <p> <s> 地球 人造 的 麦田 怪圈 </s> </p> </d>']

### deprecated code
vocab = dict()
mark = ('<s>', '</s>', '<p>', '</p>', '<d>', '</d>', '<UNK>')
pt = re.compile(r'(<\/{0,1}[sdp]>)|(<UNK>)')
pt_num = re.compile(r'\d')
for k in mark:
    vocab[k] = 0
t = time.time()
for i, s in enumerate(corpus[:], start=1):
    for k in mark:
        vocab[k] += s.count(k)
    s = pt.sub('', s)
    for w in jieba.cut(s):
        if w not in vocab:
            vocab[w] = 0
        vocab[w] += 1
    if i % 1000 == 0:
        p = i / len(corpus)*100
        print('\r%d/%d\t%.2f%%, %.1fs' % (i, len(corpus), p, (time.time() - t)/p * (100-p)), end='')
print('\r%d/%d\t%f%%' % (len(corpus), len(corpus), 100))

In [25]:
def segment(s, mark = ('<s>', '</s>', '<p>', '</p>', '<d>', '</d>', '<UNK>', '<PAD>'), pt = re.compile(r'(</{0,1}[sdp]>)|(<UNK>)|(<PAD>)')):
    vocab = dict()
    for k in mark:
        vocab[k] = s.count(k)
    s = pt.sub('', s)
    for w in s.split():
        if w not in vocab:
            vocab[w] = 0
        vocab[w] += 1
    return vocab

In [31]:
pool.close()
pool.terminate()
pool = mp.Pool(25)

In [28]:
res = pool.map(segment, corpus)

In [29]:
vocab = {}
for i in res:
    for k, v in i.items():
        if k in vocab:
            vocab[k] += v
        else:
            vocab[k] = v

In [30]:
with open(os.path.join(base,'converted', 'vocab.txt'), 'w') as f:
    for k, v in vocab.items():
        f.write('%s %d\n' % (k, v))